In [3]:
from langchain_groq import ChatGroq

In [5]:
llm= ChatGroq(
    temperature=0,
    groq_api_key='',
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("Neil armstrong is the first person to")
print(response.content)


walk on the Moon. Neil Armstrong became the first person to set foot on the Moon on July 20, 1969, during the Apollo 11 mission. His famous words upon stepping onto the lunar surface were: "That's one small step for man, one giant leap for mankind."


In [7]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-46976?from=job%20search%20funnel")
page_data= loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Apply for Retail Department Manager - Winter Garden

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsRetail StoresRetail Department Manager - Winter GardenOrlando, FloridaBecome a Part o

In [9]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
    """
    ###SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the
    following keys: 'role', 'experience', 'skills' and 'description'.
    Only return the valid JSON.
    ###VALID JSON(NO PREAMBLE):
    """
)
chain_extract= prompt_extract | llm
res= chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
{
  "role": "Retail Department Manager - Winter Garden",
  "experience": [
    "Three years of customer-facing retail or hospitality experience",
    "One year of leadership experience"
  ],
  "skills": [
    "Demonstrated ability to recruit, build and lead high-performing teams",
    "Proficient in Microsoft Office and retail business systems",
    "Ability to bend, squat, reach, climb a ladder and stand for extended periods of time with or without reasonable accommodations",
    "Proven ability to utilize tools for conflict resolution and employee coaching and counseling"
  ],
  "description": "As a Nike Department Manager, you’re the face of NIKE. Enjoy high-volume and a fast pace as your diverse experience and perspective helps guide customers in making the best decisions for them. You’ll work with your team to focus on customer service and get to the win the right way. You’ll lead, coach, and inspire your team to deliver the best experience for their customers while ensuri

In [11]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Retail Department Manager - Winter Garden',
 'experience': ['Three years of customer-facing retail or hospitality experience',
  'One year of leadership experience'],
 'skills': ['Demonstrated ability to recruit, build and lead high-performing teams',
  'Proficient in Microsoft Office and retail business systems',
  'Ability to bend, squat, reach, climb a ladder and stand for extended periods of time with or without reasonable accommodations',
  'Proven ability to utilize tools for conflict resolution and employee coaching and counseling'],
 'description': 'As a Nike Department Manager, you’re the face of NIKE. Enjoy high-volume and a fast pace as your diverse experience and perspective helps guide customers in making the best decisions for them. You’ll work with your team to focus on customer service and get to the win the right way. You’ll lead, coach, and inspire your team to deliver the best experience for their customers while ensuring your team members feel included and

In [13]:
type(json_res)

dict

In [17]:
import pandas as pd
df=pd.read_csv("my_portfolio.csv")
df

Techstack  \
0                React, Node.js, MongoDB   
1               Angular,.NET, SQL Server   
2      Vue.js, Ruby on Rails, PostgreSQL   
3                  Python, Django, MySQL   
4              Java, Spring Boot, Oracle   
5             Flutter, Firebase, GraphQL   
6                  WordPress, PHP, MySQL   
7                    Magento, PHP, MySQL   
8         React Native, Node.js, MongoDB   
9                  iOS, Swift, Core Data   
10       Android, Java, Room Persistence   
11             Kotlin, Android, Firebase   
12       Android TV, Kotlin, Android NDK   
13                     iOS, Swift, ARKit   
14        Cross-platform, Xamarin, Azure   
15          Backend, Kotlin, Spring Boot   
16         Frontend, TypeScript, Angular   
17    Full-stack, JavaScript, Express.js   
18  Machine Learning, Python, TensorFlow   
19               DevOps, Jenkins, Docker   

                                                Links  
0                 https://example.com/react-portfolio  
1               https://example.com/angular-portfolio  
2                   https://example.com/vue-portfolio  
3                https://example.com/python-portfolio  
4                  https://example.com/java-portfolio  
5               https://example.com/flutter-portfolio  
6             https://example.com/wordpress-portfolio  
7               https://example.com/magento-portfolio  
8          https://example.com/react-native-portfolio  
9                   https://example.com/ios-portfolio  
10              https://example.com/android-portfolio  
11       https://example.com/kotlin-android-portfolio  
12           https://example.com/android-tv-portfolio  
13               https://example.com/ios-ar-portfolio  
14              https://example.com/xamarin-portfolio  
15       https://example.com/kotlin-backend-portfolio  
16  https://example.com/typescript-frontend-portfolio  
17        https://example.com/full-stack-js-portfolio  
18            https://example.com/ml-python-portfolio  
19               https://example.com/devops-portfolio

In [19]:
import chromadb
import uuid


client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])
                       



In [21]:
results = collection.query(query_texts=["Experience in Python", "Expertise in React"], n_results=5)

# Checking if the results contain metadata and printing it
if results and "metadatas" in results:
    links = results["metadatas"]
    print(links)
else:
    print("No metadata found.")

[[{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/python-portfolio'}, {'links': 'https://example.com/magento-portfolio'}, {'links': 'https://example.com/wordpress-portfolio'}, {'links': 'https://example.com/devops-portfolio'}], [{'links': 'https://example.com/react-portfolio'}, {'links': 'https://example.com/react-native-portfolio'}, {'links': 'https://example.com/full-stack-js-portfolio'}, {'links': 'https://example.com/typescript-frontend-portfolio'}, {'links': 'https://example.com/ml-python-portfolio'}]]


In [23]:
job= json_res
job['skills']


['Demonstrated ability to recruit, build and lead high-performing teams',
 'Proficient in Microsoft Office and retail business systems',
 'Ability to bend, squat, reach, climb a ladder and stand for extended periods of time with or without reasonable accommodations',
 'Proven ability to utilize tools for conflict resolution and employee coaching and counseling']

In [25]:
prompt_email= PromptTemplate.from_template(
    """
    ###JOB DESCRIPTION:
    {job_description}

    ###INSTRUCTION:
    You are Lova, a business development executive at Beach and Coffee Co. It is an AI and software company seamless integration
    of business processes through automated technologies.
    Your job is to write a cold email to the client regarding the job mentioned above to fulfill their needs.
    also add most relevant ones from the  following links to showcase Beach and Cofee Co. portfolio: {link_list}
    Remember u are Lova,  A BDE AT BEACH AND COFFEE CO.
    dont need preamble

    ###EMAIL (NO PREAMBLE):
    """
)

chain_email= prompt_email | llm
res = chain_email.invoke({"job_description" : str(job), "link_list": links})
print(res.content)


Subject: Revolutionize Your Retail Operations with Beach and Coffee Co.

Dear Hiring Manager,

I came across the Retail Department Manager role at Nike and was impressed by the emphasis on exceptional customer service and team leadership. As a Business Development Executive at Beach and Coffee Co., I believe our expertise in AI and software solutions can help elevate your retail operations to the next level.

Our company specializes in seamless integration of business processes through automated technologies. We've worked with various clients to streamline their operations, enhance customer experience, and increase efficiency. I'd like to highlight a few relevant projects from our portfolio that demonstrate our capabilities:

* Our expertise in Python development can help you build custom retail management systems, as seen in our Python portfolio: https://example.com/python-portfolio
* We've also developed Magento solutions for e-commerce clients, which can be applied to your retail op